# Project Summary

**AI-Powered Financial Chatbot**

This project showcases an interactive AI-powered chatbot designed to answer financial questions using structured data 
which I extracted from 10-K Forms for the years 2022 to 2024 of the companies Tesla, Apple and Microsoft filed with 
United States Security and Exchange Commission.
Forms reflect information about Annual Financial Reports and I used that financial data to build a chatbot that is efficient and 
smart enough to answer any financial metric a user asks.

**Project Objectives**
- Analyze financial metrics such as Revenue, Net Income, Cash Flow, etc.
- Support natural language queries with fuzzy matching and context-aware logic.
- Enable year-wise and company-wise comparisons.
- Provide visual insights using charts.
- Deploy an intuitive console or web interface for user interaction.

**Tools & Technologies**
- Python (Pandas, Regex, Difflib, plotly, Gradio, Matplotlib)
- Jupyter Notebook
- Git & GitHub
- Hugging Face (for optional deployment)
- Financial Reports (US SEC 10-K Filings)

> ⚙️ This notebook demonstrates the complete step by step pipeline — from data loading, analyzing, preparing, improving code for better and better 
output to finally building a smart chatbot that can answer business questions like:
> - *“What was Tesla’s revenue in 2023?”*
> - *“Compare Net Income of Apple and Microsoft in 2022”*
> - *“Show me the trend of R&D expense for Tesla”*

# Importing Libraries:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Loading and Reviewing Dataset:

In [2]:
# Load the CSV file
df = pd.read_csv('chatbot_cleaned.csv')

In [3]:
# Show the first 5 rows
df.head()

,Metric,2024,2023,2022,Company
0,Total Revenue,97000.0,96773.0,81462.0,Tesla
1,Net Income,7050.0,14993.0,12556.0,Tesla
2,Operating Income,8880.0,13666.0,13666.0,Tesla
3,R&D Expense,4450.0,3969.0,3080.0,Tesla
4,Total Assets,122570.0,106620.0,82338.0,Tesla


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Metric   40 non-null     object 
 1   2024     40 non-null     float64
 2   2023     40 non-null     float64
 3   2022     40 non-null     float64
 4   Company  40 non-null     object 
dtypes: float64(3), object(2)
memory usage: 1.7+ KB


In [5]:
df.describe()

,2024,2023,2022
count,40.000000,40.000000,40.000000
mean,100244.473750,90422.833000,84484.997750
std,123001.568635,108445.241549,105672.925276
min,2.200000,4.300000,3.620000
25%,9305.250000,12906.750000,12193.000000
50%,64892.500000,61798.000000,49206.500000
75%,122731.500000,110135.750000,104747.000000
max,511920.000000,410530.000000,394328.000000


# Cleaning & Reshaping Data for Easy Access:

In [6]:
# Melt the dataframe so year becomes a column
df_long = df.melt(id_vars=['Metric', 'Company'], 
                  value_vars=['2022', '2023', '2024'],
                  var_name='Year',
                  value_name='Value')

# Clean: lowercase everything for matching, and strip spaces
df_long['Metric_clean'] = df_long['Metric'].str.lower().str.strip()
df_long['Company_clean'] = df_long['Company'].str.lower().str.strip()
df_long['Year'] = df_long['Year'].astype(str)

In [7]:
df_long.head()

,Metric,Company,Year,Value,Metric_clean,Company_clean
0,Total Revenue,Tesla,2022,81462.0,total revenue,tesla
1,Net Income,Tesla,2022,12556.0,net income,tesla
2,Operating Income,Tesla,2022,13666.0,operating income,tesla
3,R&D Expense,Tesla,2022,3080.0,r&d expense,tesla
4,Total Assets,Tesla,2022,82338.0,total assets,tesla


# Starting Building Process:

**Adding Fuzzy Matching & Entity Extraction**

This function will take a user query like:
*"Who had the highest revenue in 2023?"*

In [8]:
# Initial alias mapping for common terms
metric_aliases = {
    "eps": "eps (basic)",
    "basic eps": "eps (basic)",
    "diluted eps": "eps (diluted)",
    "cash flow": "free cash flow (est.)",
    "operating cash": "operating cash flow",
    "free cash": "free cash flow (est.)",
    "research": "r&d expense",
    "r&d": "r&d expense",
    "revenue": "total revenue",
    "income": "net income",
    "profit": "net income"
}

**Defining mathces and metrics**

In [9]:
from rapidfuzz import process, fuzz

# Unique lists
all_metrics = df_long['Metric_clean'].unique().tolist()
all_companies = df_long['Company_clean'].unique().tolist()
all_years = ['2022', '2023', '2024']

def fuzzy_match(text, choices, label):
    match, score, _ = process.extractOne(text.lower(), choices, scorer=fuzz.partial_ratio)
    print(f"Matching {label}: '{text}' → '{match}' (score: {score})")
    return match if score >= 60 else None

def extract_entities(question):
    # Match year
    year = next((y for y in all_years if y in question), None)

    # Try to match entire question to company and metric
    company = fuzzy_match(question, all_companies, "company")
    metric = fuzzy_match(question, all_metrics, "metric")

    return company, metric, year

# Updating code to Improve output & Functionality:
**Combining aliases and defined metrics**

In [10]:
from rapidfuzz import process, fuzz

# Step 1: Define valid values from the dataframe
all_metrics = df_long['Metric_clean'].unique().tolist()
all_companies = df_long['Company_clean'].unique().tolist()
all_years = ['2022', '2023', '2024']

# Step 2: Define common aliases for metrics
metric_aliases = {
    "eps": "eps (basic)",
    "basic eps": "eps (basic)",
    "diluted eps": "eps (diluted)",
    "cash flow": "free cash flow (est.)",
    "operating cash": "operating cash flow",
    "free cash": "free cash flow (est.)",
    "research": "r&d expense",
    "r&d": "r&d expense",
    "revenue": "total revenue",
    "income": "net income",
    "profit": "net income",
    "cloud": "cloud revenue",
    "capital expense": "capital expenditures",
    "capex": "capital expenditures",
    "assets": "total assets",
    "liabilities": "total liabilities",
    "equity": "shareholders' equity"
}

# Step 3: Fuzzy match helper
def fuzzy_match(text, choices, label):
    match, score, _ = process.extractOne(text.lower(), choices, scorer=fuzz.partial_ratio)
    print(f"Matching {label}: '{text}' → '{match}' (score: {score:.1f})")
    return match if score >= 60 else None

# Step 4: Main extractor function
def extract_entities(question):
    question_lower = question.lower()

    # Match year directly
    year = next((y for y in all_years if y in question_lower), None)

    # Alias check for metric
    metric = None
    for alias, standard in metric_aliases.items():
        if alias in question_lower:
            metric = standard
            print(f"Alias matched metric: '{alias}' → '{standard}'")
            break

    # If alias didn't hit, fall back to fuzzy match
    if not metric:
        metric = fuzzy_match(question, all_metrics, "metric")

    # Always fuzzy match the company
    company = fuzzy_match(question, all_companies, "company")

    return company, metric, year

**CHECKING Function** **"Extract Entities"**

In [11]:
extract_entities("How much did Microsoft earn in revenue in 2023?")

Alias matched metric: 'revenue' → 'total revenue'
Matching company: 'How much did Microsoft earn in revenue in 2023?' → 'microsoft' (score: 100.0)


('microsoft', 'total revenue', '2023')

# Building the Chatbot Loop: 
**This loop will:** Allow user to ask a financial question

In [12]:
def chatbot():
    print("💬 Financial Chatbot is ready! Ask about Tesla, Apple, or Microsoft (2022–2024). Type 'exit' to stop.")
    
    while True:
        question = input("\nAsk your question: ")
        if question.lower() == 'exit':
            print("👋 Chatbot session ended.")
            break
        
        company, metric, year = extract_entities(question)

        if not all([company, metric, year]):
            print("❌ Sorry, I couldn't understand your question. Please try again with a metric, company, and year.")
            continue
        
        # Filter the value
        match = df_long[
            (df_long['Company_clean'] == company) &
            (df_long['Metric_clean'] == metric) &
            (df_long['Year'] == year)
        ]

        if match.empty:
            print("❌ Sorry, I couldn't find data for that combination.")
        else:
            value = match['Value'].values[0]
            formatted = f"${value:,.0f} million"
            print(f"📊 {match['Company'].values[0]} had *{match['Metric'].values[0]}* of **{formatted}** in {year}.")

In [13]:
chatbot()

💬 Financial Chatbot is ready! Ask about Tesla, Apple, or Microsoft (2022–2024). Type 'exit' to stop.



Ask your question:  exit


👋 Chatbot session ended.


**Key Findings:**
- Tested chatbot with more than 20 different questions and it showed a bit satisfying results.

**Types of Tested Questions:**
- General performance comparisons
- Individual company and metric
- Highest/Lowest across all Companies

# Defining Comparative Logic/Detecting Comparative Intent:

***I will:***

**1: Detect if the user’s question is comparative (e.g., "which company", "who had more", etc.)**

In [14]:
def is_comparative_question(question):
    patterns = ['which company', 'who had', 'who earned', 'which firm', 'which one', 'highest', 'most', 'lowest', 'least']
    return any(p in question.lower() for p in patterns)

**2: Defining Comparative Answering Function**

In [15]:
def answer_comparative_question(question):
    question_lower = question.lower()

    # Extract year
    year = next((y for y in all_years if y in question_lower), None)
    if not year:
        print("❌ Please include a year (e.g., 2023) in your question.")
        return

    # Alias-based metric extraction
    metric = None
    for alias, standard in metric_aliases.items():
        if alias in question_lower:
            metric = standard
            print(f"Alias matched metric: '{alias}' → '{standard}'")
            break

    if not metric:
        metric = fuzzy_match(question, all_metrics, "metric")

    if not metric:
        print("❌ Couldn't determine what metric you're asking about.")
        return

    # Filter relevant rows
    matches = df_long[(df_long['Metric_clean'] == metric) & (df_long['Year'] == year)]

    if matches.empty:
        print("❌ No data available for that metric and year.")
        return

    # Sort by value descending
    sorted_matches = matches.sort_values(by='Value', ascending=False).reset_index(drop=True)

    # Top result
    top = sorted_matches.iloc[0]
    formatted_value = f"${top['Value']:,.0f} million"
    print(f"🏆 *{top['Company']}* had the highest {top['Metric']} in {year}: **{formatted_value}**")

    # Optional: Print all companies
    print("\n📊 All companies ranked:")
    for _, row in sorted_matches.iterrows():
        val = f"${row['Value']:,.0f} million"
        print(f"- {row['Company']}: {val}")

**3: Updating chatbot to Handle questioning Situations**

In [16]:
def chatbot():
    print("💬 Financial Chatbot is ready! Ask about Tesla, Apple, or Microsoft (2022–2024). Type 'exit' to stop.")
    
    while True:
        question = input("\nAsk your question: ")
        if question.lower() == 'exit':
            print("👋 Chatbot session ended.")
            break

        # Check if it's a comparative question
        if is_comparative_question(question):
            answer_comparative_question(question)
            continue

        # Regular (non-comparative) question
        company, metric, year = extract_entities(question)

        if not all([company, metric, year]):
            print("❌ Sorry, I couldn't understand your question. Please try again with a metric, company, and year.")
            continue
        
        # Filter the value
        match = df_long[
            (df_long['Company_clean'] == company) &
            (df_long['Metric_clean'] == metric) &
            (df_long['Year'] == year)
        ]

        if match.empty:
            print("❌ Sorry, I couldn't find data for that combination.")
        else:
            value = match['Value'].values[0]
            formatted = f"${value:,.0f} million"
            print(f"📊 {match['Company'].values[0]} had *{match['Metric'].values[0]}* of **{formatted}** in {year}.")

In [17]:
chatbot()

💬 Financial Chatbot is ready! Ask about Tesla, Apple, or Microsoft (2022–2024). Type 'exit' to stop.



Ask your question:  exit


👋 Chatbot session ended.


**Key Findings**
- Tested more than 15 different questions and got improved output which means
the chatbot is working better than before.

# Making Improvements
**4: Upgrading answer_comparative_question() with Low/High Detection to get more accuracy**

In [18]:
def answer_comparative_question(question):
    question_lower = question.lower()

    # Detect year
    year = next((y for y in all_years if y in question_lower), None)
    if not year:
        print("❌ Please include a year (e.g., 2023) in your question.")
        return

    # Detect metric
    metric = None
    for alias, standard in metric_aliases.items():
        if alias in question_lower:
            metric = standard
            print(f"Alias matched metric: '{alias}' → '{standard}'")
            break
    if not metric:
        metric = fuzzy_match(question, all_metrics, "metric")
    if not metric:
        print("❌ Couldn't determine what metric you're asking about.")
        return

    # Get matching values
    matches = df_long[(df_long['Metric_clean'] == metric) & (df_long['Year'] == year)]
    if matches.empty:
        print("❌ No data available for that metric and year.")
        return

    # Detect intent: highest or lowest
    is_lowest = any(x in question_lower for x in ['lowest', 'least'])

    # Sort accordingly
    sorted_matches = matches.sort_values(by='Value', ascending=is_lowest).reset_index(drop=True)
    top = sorted_matches.iloc[0]
    val_formatted = f"${top['Value']:,.0f} million"

    if is_lowest:
        print(f"📉 *{top['Company']}* had the lowest {top['Metric']} in {year}: **{val_formatted}**")
    else:
        print(f"🏆 *{top['Company']}* had the highest {top['Metric']} in {year}: **{val_formatted}**")

    # Show all
    print("\n📊 All companies ranked:")
    for _, row in sorted_matches.iterrows():
        value_str = f"${row['Value']:,.0f} million"
        print(f"- {row['Company']}: {value_str}")

**5: Implementing the highest/lowest company detection feature by combining the steps taken before**

**This feature will:**
- Detect if the user asks for the highest or lowest company by a metric in a specific year.
- Extract the metric and year using fuzzy matching and aliases (as already implemented).
- Return a ranked list of all companies for that metric in the given year.
- Clearly highlight the company with the highest or lowest value

In [19]:
import pandas as pd
import numpy as np
from rapidfuzz import process, fuzz

# Load cleaned dataset
df = pd.read_csv("chatbot_cleaned.csv")

# Melt to long format for easier querying
df_long = pd.melt(df, id_vars=['Metric', 'Company'], var_name='Year', value_name='Value')
df_long['Year'] = df_long['Year'].astype(int)
df_long['Metric_clean'] = df_long['Metric'].str.lower().str.strip()
df_long['Company_clean'] = df_long['Company'].str.lower().str.strip()

# Define aliases
company_aliases = {'apple': 'apple', 'aapl': 'apple', 'msft': 'microsoft', 'microsoft': 'microsoft', 'tesla': 'tesla', 'tsla': 'tesla'}
metric_aliases = {
    'revenue': 'total revenue',
    'income': 'net income',
    'profit': 'net income',
    'operating profit': 'operating income',
    'operating income': 'operating income',
    'r&d': 'r&d expense',
    'research': 'r&d expense',
    'assets': 'total assets',
    'liabilities': 'total liabilities',
    'equity': "shareholders' equity",
    'cash': 'cash & cash equivalents',
    'cash flow': 'free cash flow (est.)',
    'free cash flow': 'free cash flow (est.)',
    'capex': 'capital expenditures',
    'capital expenditure': 'capital expenditures',
    'eps': 'eps (basic)',
    'eps basic': 'eps (basic)',
    'eps diluted': 'eps (diluted)',
    'cloud': 'cloud revenue'
}

# Entity extraction
def extract_entities(text):
    text_lower = text.lower()

    # Alias matching
    metric_key = next((alias for alias in metric_aliases if alias in text_lower), None)
    matched_metric = metric_aliases.get(metric_key) if metric_key else None
    if matched_metric:
        print(f"Alias matched metric: '{metric_key}' → '{matched_metric}'")

    # Fuzzy match company
    company_match = process.extractOne(text_lower, df_long['Company_clean'].unique(), scorer=fuzz.token_sort_ratio)
    company = company_match[0] if company_match else None

    # Year matching
    year = next((int(word) for word in text.split() if word.isdigit() and int(word) in df_long['Year'].unique()), None)

    return company, matched_metric, year

# Main chatbot loop
def chatbot():
    print("💬 Financial Chatbot is ready! Ask about Tesla, Apple, or Microsoft (2022–2024). Type 'exit' to stop.")
    
    while True:
        question = input("\nAsk your question: ")
        if question.lower() == 'exit':
            print("👋 Chatbot session ended.")
            break

        company, metric, year = extract_entities(question)

        if not metric or not year:
            print("❌ Please include both a financial metric and year.")
            continue

        # Filter data
        subset = df_long[(df_long['Metric_clean'] == metric) & (df_long['Year'] == year)]
        if subset.empty:
            print("❌ No data found for that metric and year.")
            continue

        # Handle comparative queries first
        q_lower = question.lower()
        if any(keyword in q_lower for keyword in ['highest', 'most', 'top']):
            best = subset.loc[subset['Value'].idxmax()]
            print(f"🏆 *{best['Company']}* had the highest {best['Metric']} in {year}: **${best['Value']:,.0f} million**\n")
        elif any(keyword in q_lower for keyword in ['lowest', 'least', 'bottom']):
            worst = subset.loc[subset['Value'].idxmin()]
            print(f"📉 *{worst['Company']}* had the lowest {worst['Metric']} in {year}: **${worst['Value']:,.0f} million**\n")
        elif company:
            match = subset[subset['Company_clean'] == company]
            if not match.empty:
                val = match['Value'].values[0]
                print(f"📊 {match['Company'].values[0]} had *{match['Metric'].values[0]}* of **${val:,.0f} million** in {year}.")
            else:
                print("❌ No data found for that company/metric/year.")

        # Always show full ranking
        subset_sorted = subset.sort_values(by='Value', ascending=False)
        print("\n📊 All companies ranked:")
        for _, row in subset_sorted.iterrows():
            print(f"- {row['Company']}: ${row['Value']:,.0f} million")

In [20]:
chatbot()

💬 Financial Chatbot is ready! Ask about Tesla, Apple, or Microsoft (2022–2024). Type 'exit' to stop.



Ask your question:  exit


👋 Chatbot session ended.


**Key Findings**

After testing chatbot with dynamic questions it is found that much improvement is still needed to make our chatbot smarter.

**6: Making chatbot smarter with chart integration and addressing remaining metrics**

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from rapidfuzz import process, fuzz

# Load your cleaned dataset
df = pd.read_csv('chatbot_cleaned.csv')

# Preprocessing
df_long = df.melt(id_vars=["Metric", "Company"], var_name="Year", value_name="Value")
df_long["Year"] = df_long["Year"].astype(int)
df_long["Metric_clean"] = df_long["Metric"].str.lower().str.strip()
df_long["Company_clean"] = df_long["Company"].str.lower().str.strip()

# Alias dictionary for fuzzy mapping
alias_dict = {
    'revenue': 'total revenue',
    'income': 'net income',
    'profit': 'net income',
    'operating profit': 'operating income',
    'cash': 'cash & cash equivalents',
    'cash flow': 'free cash flow (est.)',
    'r&d': 'r&d expense',
    'assets': 'total assets',
    'liabilities': 'total liabilities',
    'equity': "shareholders' equity",
    'eps': 'eps (basic)'
}

def match_alias(metric):
    for key, value in alias_dict.items():
        if key in metric.lower():
            print(f"Alias matched metric: '{key}' → '{value}'")
            return value
    return metric

# 🎯 Extract entities from question
import re

def extract_entities(text):
    text_lower = text.lower()
    
    # Handle known aliases
    for key, val in metric_aliases.items():
        if key in text_lower:
            matched_metric = val
            print(f"Alias matched metric: '{key}' → '{matched_metric}'")
            break
    else:
        matched_metric = None
    
    # Fuzzy match company
    company_match = process.extractOne(text_lower, df_long['Company_clean'].unique(), scorer=fuzz.token_sort_ratio)
    company = company_match[0] if company_match else None
    print(f"Matching company: '{text}' → '{company}' (score: {company_match[1] if company_match else 'N/A'})")

    # Fuzzy match metric if no alias matched
    if not matched_metric:
        metric_match = process.extractOne(text_lower, df_long['Metric_clean'].unique(), scorer=fuzz.token_sort_ratio)
        matched_metric = metric_match[0] if metric_match else None
        print(f"Matching metric: '{text}' → '{matched_metric}' (score: {metric_match[1] if metric_match else 'N/A'})")

    # Extract year using regex
    match_year = re.search(r'\b(2022|2023|2024)\b', text)
    year = int(match_year.group(0)) if match_year else None

    return company, matched_metric, year

# 📊 Plot bar chart using Plotly
def plot_bar_chart(sub_df, metric, year):
    sorted_df = sub_df.sort_values(by="Value", ascending=False)
    fig = go.Figure(data=[
        go.Bar(x=sorted_df['Company'], y=sorted_df['Value'], text=sorted_df['Value'], textposition='auto')
    ])
    fig.update_layout(
        title=f"{metric.title()} Comparison ({year})",
        xaxis_title="Company",
        yaxis_title=metric.title(),
        template="plotly_dark",
        height=400
    )
    fig.show()

# 💬 Main chatbot loop
def chatbot():
    print("💬 Financial Chatbot is ready! Ask about Tesla, Apple, or Microsoft (2022–2024). Type 'exit' to stop.\n")
    
    while True:
        question = input("Ask your question:  ")
        if question.lower() == 'exit':
            print("👋 Chatbot session ended.")
            break
        
        company, metric, year = extract_entities(question)
        
        if not metric or not year:
            print("❌ Please include both a financial metric and year.\n")
            continue

        metric_cleaned = metric.lower().strip()
        sub_df = df_long[(df_long["Metric_clean"] == metric_cleaned) & (df_long["Year"] == year)]
        
        if sub_df.empty:
            print(f"⚠️ No data found for {metric.title()} in {year}.\n")
            continue

        # Specific company query
        if company:
            comp_val = sub_df[sub_df["Company_clean"] == company]
            if not comp_val.empty:
                val = comp_val.iloc[0]["Value"]
                print(f"📊 {comp_val.iloc[0]['Company']} had *{metric.title()}* of **${val:,.0f} million** in {year}.\n")

        # Ranking info
        sorted_df = sub_df.sort_values(by="Value", ascending=False)
        if "highest" in question.lower() or "most" in question.lower():
            top = sorted_df.iloc[0]
            print(f"🏆 *{top['Company']}* had the highest {metric.title()} in {year}: **${top['Value']:,.0f} million**\n")
        elif "lowest" in question.lower() or "least" in question.lower():
            bottom = sorted_df.iloc[-1]
            print(f"📉 *{bottom['Company']}* had the lowest {metric.title()} in {year}: **${bottom['Value']:,.0f} million**\n")

        # Display all companies' values
        print("📊 All companies ranked:")
        for _, row in sorted_df.iterrows():
            print(f"- {row['Company']}: ${row['Value']:,.0f} million")
        print()

        # Show interactive chart
        plot_bar_chart(sub_df, metric, year)

In [22]:
chatbot()

💬 Financial Chatbot is ready! Ask about Tesla, Apple, or Microsoft (2022–2024). Type 'exit' to stop.



Ask your question:   exit


👋 Chatbot session ended.


**Key Findings**
Chatbot is working better but still needs improvement.

**7: Making fixes like alias mapping for better results**

In [23]:
import pandas as pd
from fuzzywuzzy import process, fuzz
import matplotlib.pyplot as plt

# Load data
df = pd.read_csv("chatbot_cleaned.csv")

# Convert to long format
df_long = df.melt(id_vars=["Metric", "Company"], var_name="Year", value_name="Value")
df_long["Year"] = df_long["Year"].astype(int)
df_long.columns = ["metric", "company", "year", "value"]

# Clean columns for fuzzy matching
df_long["metric_clean"] = df_long["metric"].str.lower().str.strip()
df_long["company_clean"] = df_long["company"].str.lower().str.strip()

# Smarter alias matcher
def match_metric_alias(text):
    alias_map = {
        "revenue": "total revenue",
        "sales": "total revenue",
        "sold": "total revenue",
        "income": "net income",
        "profit": "net income",
        "earned": "net income",
        "made": "net income",
        "cash": "cash & cash equivalents",
        "cash flow": "free cash flow (est.)",
        "r&d": "r&d expense",
        "research": "r&d expense",
        "assets": "total assets",
        "liabilities": "total liabilities",
        "equity": "shareholders’ equity",
        "opex": "operating income",
        "operating income": "operating income",
        "eps": "eps (basic)",
        "cloud": "cloud revenue",
        "free cash flow": "free cash flow (est.)",
    }

    for alias, metric in alias_map.items():
        if alias in text:
            print(f"Alias matched metric: '{alias}' → '{metric}'")
            return metric, alias
    return None, None

# Extract entities
def extract_entities(text):
    text_lower = text.lower()

    matched_metric, alias_key = match_metric_alias(text_lower)

    if not matched_metric:
        matched_metric, score = process.extractOne(
            text_lower, df_long["metric_clean"].unique(), scorer=fuzz.partial_ratio
        )

    matched_company, score = process.extractOne(
        text_lower, df_long["company_clean"].unique(), scorer=fuzz.token_sort_ratio
    )

    year = next((int(w) for w in text.split() if w.isdigit() and int(w) in df_long["year"].unique()), None)

    print(f"Matching company: '{text}' → '{matched_company}'")
    print(f"Matching metric: '{text}' → '{matched_metric}'")
    return matched_company, matched_metric, year

# Generate bar chart
def plot_comparison(metric, year, df):
    subset = df[(df["metric_clean"] == metric.lower()) & (df["year"] == year)]
    if subset.empty:
        print("⚠️ No data found to plot.")
        return

    plt.figure(figsize=(7, 4))
    plt.bar(subset["company"], subset["value"], color=["#1f77b4", "#ff7f0e", "#2ca02c"])
    plt.title(f"{metric.title()} in {year}")
    plt.ylabel("Value (in millions)")
    plt.xlabel("Company")
    plt.tight_layout()
    plt.grid(axis="y", linestyle="--", alpha=0.5)
    plt.show()

# Chatbot loop
print("💬 Financial Chatbot is ready! Ask about Tesla, Apple, or Microsoft (2022–2024). Type 'exit' to stop.\n")

while True:
    user_input = input("Ask your question:   ")
    if user_input.lower() in ["exit", "quit", "bye"]:
        print("👋 Exiting chatbot. Have a great day!")
        break

    company, metric, year = extract_entities(user_input)

    if not metric:
        print("❌ Could not detect a financial metric. Try mentioning revenue, profit, or cash flow.\n")
        continue
    if not year:
        print("❌ Could not detect a year (2022–2024). Please specify the year.\n")
        continue

    # Get relevant data
    filtered = df_long[
        (df_long["company_clean"] == company.lower()) &
        (df_long["metric_clean"] == metric.lower()) &
        (df_long["year"] == year)
    ]

    if not filtered.empty:
        value = filtered["value"].values[0]
        print(f"📊 {company.title()} had *{metric.title()}* of **${value:,} million** in {year}.\n")

        # Show full ranking
        compare_df = df_long[
            (df_long["metric_clean"] == metric.lower()) &
            (df_long["year"] == year)
        ].sort_values(by="value", ascending=False)

        print("📊 All companies ranked:")
        for _, row in compare_df.iterrows():
            print(f"- {row['company']}: ${row['value']:,}")

        # Optional: show percent difference between top 2
        if len(compare_df) >= 2:
            top1 = compare_df.iloc[0]
            top2 = compare_df.iloc[1]
            pct_diff = (top1["value"] - top2["value"]) / top2["value"] * 100
            print(f"\n📈 {top1['company'].title()} earned {pct_diff:.1f}% more than {top2['company'].title()}.\n")

        # Plot chart
        plot_comparison(metric, year, df_long)

    else:
        print("❌ Data not found. Check company name, metric, or year.\n")

💬 Financial Chatbot is ready! Ask about Tesla, Apple, or Microsoft (2022–2024). Type 'exit' to stop.



Ask your question:    exit


👋 Exiting chatbot. Have a great day!


**Lower casing data to avoid inaccuracy**

In [25]:
# normalize metric column once 
df['Metric'] = df['Metric'].str.lower()

**8: Making more fixes and updating code to add ***GRADIO INTERFACE*** for better UX**

In [26]:
import pandas as pd
import re
import plotly.express as px
import gradio as gr

# Load data
df = pd.read_csv("chatbot_cleaned.csv")

# Clean metric names to lowercase
metrics_list = df['Metric'].str.lower().unique()

# Create alias dictionary (comprehensive)
alias_dict = {
    "revenue": "total revenue",
    "sales": "total revenue",
    "income": "net income",
    "profit": "net income",
    "earnings": "net income",
    "operating profit": "operating income",
    "operating income": "operating income",
    "r&d": "r&d expense",
    "rnd": "r&d expense",
    "research": "r&d expense",
    "research and development": "r&d expense",
    "r&d est": "r&d expense (est.)",
    "r&d estimate": "r&d expense (est.)",
    "assets": "total assets",
    "liabilities": "total liabilities",
    "debt": "total liabilities",
    "equity": "shareholders' equity",
    "shareholder equity": "shareholders' equity",
    "cash": "cash & cash equivalents",
    "cash equivalents": "cash & cash equivalents",
    "cash flow": "operating cash flow",
    "ocf": "operating cash flow",
    "capex": "capital expenditures",
    "capital expense": "capital expenditures",
    "capital expenditure": "capital expenditures",
    "free cash": "free cash flow (est.)",
    "free cash flow": "free cash flow (est.)",
    "fcf": "free cash flow (est.)",
    "eps": "eps (basic)",
    "eps basic": "eps (basic)",
    "eps diluted": "eps (diluted)",
    "cloud": "cloud revenue",
    "cloud revenue": "cloud revenue",
}

# Plotting functions
def plot_bar_chart(company_df, metric, year):
    fig = px.bar(company_df, x='Company', y=str(year), title=f"{metric.title()} by Company in {year}")
    fig.update_layout(template='plotly_white')
    return fig

def plot_comparison_chart(metric, year):
    metric_df = df[df['Metric'].str.lower() == metric.lower()]
    fig = px.bar(metric_df, x='Company', y=str(year), title=f"{metric.title()} Comparison in {year}")
    fig.update_layout(template='plotly_white')
    return fig

# Chatbot logic
def chatbot(message):
    message = message.lower()

    # Match company and year
    company_match = re.findall(r"(tesla|apple|microsoft)", message)
    year_match = re.findall(r"(2022|2023|2024)", message)

    # Match metric directly from metric list
    metric_match = [metric for metric in metrics_list if metric in message]

    # If no direct match, try alias dictionary
    if not metric_match:
        for alias, standard in alias_dict.items():
            if alias in message:
                metric_match = [standard]
                break

    company = company_match[0] if company_match else None
    year = int(year_match[0]) if year_match else 2024
    metric = metric_match[0] if metric_match else None

    # Specific company + metric (optionally year)
    if company and metric:
        value = df[(df['Company'].str.lower() == company) & 
                   (df['Metric'].str.lower() == metric)][str(year)].values
        if value.size > 0:
            response = f"📊 *{company.title()}* had **{metric.title()}** of **${value[0]:,.2f}** in {year}."
            chart = plot_bar_chart(df[df['Company'].str.lower() == company], metric, year)
            return response, chart
        else:
            return "❌ Data not found for that metric and year.", None

    # General comparison (only metric given)
    elif metric:
        metric_df = df[df['Metric'].str.lower() == metric]
        if not metric_df.empty:
            if str(year) in metric_df.columns:
                top_company = metric_df.loc[metric_df[str(year)].idxmax()]['Company']
                top_value = metric_df[str(year)].max()
                response = f"🏆 *{top_company}* had the highest **{metric.title()}** in {year}: **${top_value:,.2f}**."
                chart = plot_comparison_chart(metric, year)
                return response, chart
            else:
                return "❌ Year not found in dataset.", None
        else:
            return "❌ Metric not found in dataset.", None

    # Fallback
    else:
        return "❌ Please mention a valid financial metric (e.g. revenue, profit, liabilities, R&D, EPS, etc.).", None

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("💬 **Financial Chatbot**: Ask about Tesla, Apple, or Microsoft (2022–2024).<br><br>Examples:<br>• *Which company earned more in 2024?*<br>• *Apple revenue in 2023*", elem_id="title")

    with gr.Row():
        with gr.Column():
            msg = gr.Textbox(label="Ask your question:")
            btn = gr.Button("Submit")
            chat_output = gr.Textbox(label="Response")
        chart_output = gr.Plot(label="Chart")

    btn.click(fn=chatbot, inputs=msg, outputs=[chat_output, chart_output])

# Launch
demo.launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


**Key Findings:**
- After testing, encountered issues regarding questions related to comparison of companies.

*e.g (Which company earned more in 2024?) did not give any result.*

**9: Making more fixes encountered in the code above and adding details on UI**

In [27]:
import pandas as pd
import re
import plotly.express as px
import gradio as gr

# Load data
df = pd.read_csv("chatbot_cleaned.csv")

# Clean metric names to lowercase
metrics_list = df['Metric'].str.lower().unique()

# Expanded alias dictionary
alias_dict = {
    "revenue": "total revenue",
    "sales": "total revenue",
    "income": "net income",
    "profit": "net income",
    "earnings": "net income",
    "earned": "net income",
    "earned more": "net income",
    "made profit": "net income",
    "made money": "net income",
    "make money": "net income",
    "made more": "net income",
    "make more": "net income",
    "who earned more": "net income",
    "which company earned more": "net income",
    "which earned": "net income",
    "operating profit": "operating income",
    "operating income": "operating income",
    "r&d": "r&d expense",
    "rnd": "r&d expense",
    "research": "r&d expense",
    "research and development": "r&d expense",
    "r&d est": "r&d expense (est.)",
    "r&d estimate": "r&d expense (est.)",
    "assets": "total assets",
    "liabilities": "total liabilities",
    "debt": "total liabilities",
    "equity": "shareholders' equity",
    "shareholder equity": "shareholders' equity",
    "cash": "cash & cash equivalents",
    "cash equivalents": "cash & cash equivalents",
    "cash flow": "operating cash flow",
    "ocf": "operating cash flow",
    "capex": "capital expenditures",
    "capital expense": "capital expenditures",
    "capital expenditure": "capital expenditures",
    "free cash": "free cash flow (est.)",
    "free cash flow": "free cash flow (est.)",
    "fcf": "free cash flow (est.)",
    "eps": "eps (basic)",
    "eps basic": "eps (basic)",
    "eps diluted": "eps (diluted)",
    "cloud": "cloud revenue",
    "cloud revenue": "cloud revenue",
}

# Chart functions
def plot_bar_chart(company_df, metric, year):
    fig = px.bar(company_df, x='Company', y=str(year), title=f"{metric.title()} by Company in {year}", text=str(year))
    fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
    fig.update_layout(template='plotly_white', uniformtext_minsize=8, uniformtext_mode='show')
    return fig

def plot_comparison_chart(metric, year):
    metric_df = df[df['Metric'].str.lower() == metric.lower()]
    fig = px.bar(metric_df, x='Company', y=str(year), title=f"{metric.title()} Comparison in {year}", text=str(year))
    fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
    fig.update_layout(template='plotly_white', uniformtext_minsize=8, uniformtext_mode='show')
    return fig

# Chatbot logic
def chatbot(message):
    message = message.lower()

    # Match companies and year
    companies = re.findall(r"(tesla|apple|microsoft)", message)
    year_match = re.findall(r"(2022|2023|2024)", message)
    year = int(year_match[0]) if year_match else 2024

    # Match metric
    metric_match = [metric for metric in metrics_list if metric in message]
    if not metric_match:
        for alias, standard in alias_dict.items():
            if re.search(rf"\b{re.escape(alias)}\b", message):
                metric_match = [standard]
                break
    metric = metric_match[0] if metric_match else None

    # Case 1: Multiple companies comparison
    if companies and metric:
        responses = []
        comparison_df = df[(df['Company'].str.lower().isin(companies)) & 
                           (df['Metric'].str.lower() == metric)]
        for company in companies:
            value = comparison_df[(comparison_df['Company'].str.lower() == company)][str(year)].values
            if value.size > 0:
                responses.append(f"*{company.title()}*: **${value[0]:,.2f}**")
        if responses:
            response = f"📊 **{metric.title()} in {year}**:\n" + "\n".join(responses)
            chart = plot_bar_chart(comparison_df, metric, year)
            return response, chart
        else:
            return "❌ Data not found for given companies/metric/year.", None

    # Case 2: Metric only (general comparison)
    elif metric:
        metric_df = df[df['Metric'].str.lower() == metric]
        if not metric_df.empty and str(year) in metric_df.columns:
            top_company = metric_df.loc[metric_df[str(year)].idxmax()]['Company']
            top_value = metric_df[str(year)].max()
            response = f"🏆 *{top_company}* had the highest **{metric.title()}** in {year}: **${top_value:,.2f}**."
            chart = plot_comparison_chart(metric, year)
            return response, chart
        else:
            return "❌ Metric or year not found in dataset.", None

    # Case 3: Company only → assume Net Income
    elif companies:
        responses = []
        comparison_df = df[(df['Company'].str.lower().isin(companies)) & 
                           (df['Metric'].str.lower() == "net income")]
        for company in companies:
            value = comparison_df[(comparison_df['Company'].str.lower() == company)][str(year)].values
            if value.size > 0:
                responses.append(f"*{company.title()}*: **${value[0]:,.2f}** (Net Income)")
        if responses:
            response = f"📊 **Net Income in {year}**:\n" + "\n".join(responses)
            chart = plot_bar_chart(comparison_df, "net income", year)
            return response, chart
        else:
            return "❌ Could not find Net Income data.", None

    else:
        return "❌ Please mention a valid company or financial metric (e.g. revenue, EPS, R&D).", None

# Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown(
    """
    # 💬 Financial Chatbot  
    👨‍💻 Developed by **Bilal Hassan Nizami**  

    Ask about Tesla, Apple, or Microsoft (2022–2024).  

    #### 📌 Example Questions:
    • *Which company earned more in 2024?*  
    • *Apple revenue in 2023*  
    """,
    elem_id="title"
)

    with gr.Row():
        with gr.Column():
            msg = gr.Textbox(label="Ask your question:")
            btn = gr.Button("Submit")
            chat_output = gr.Textbox(label="Response")
        chart_output = gr.Plot(label="Chart")

    btn.click(fn=chatbot, inputs=msg, outputs=[chat_output, chart_output])

# ✅ Launch with public link enabled
demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7862
* Running on public URL: https://0db7e35f9e3a14ba2d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


**Key Findings**
After testing more than 30 different questions chatbot showed much improvement and is now more likely to be used.

**10: Working more on code to have Chatbot dealing with Year-wise comparisons and Converting Response box into modern Chatting Style Conversation box**

In [35]:
import pandas as pd
import re
import plotly.express as px
import gradio as gr

# Load data
df = pd.read_csv("chatbot_cleaned.csv")

# Metric cleaning
metrics_list = df['Metric'].str.lower().unique()
all_years = [2022, 2023, 2024]
default_companies = ["tesla", "apple", "microsoft"]

# Aliases metrics
alias_dict = {
    "revenue": "total revenue",
    "rvnu": "total revenue",
    "revnu": "total revenue",
    "revenu": "total revenue",
    "sales": "total revenue",
    "profit": "net income",
    "earnings": "net income",
    "earned": "net income",
    "earned more": "net income",
    "made profit": "net income",
    "made money": "net income",
    "make money": "net income",
    "made more": "net income",
    "make more": "net income",
    "who earned more": "net income",
    "which company earned more": "net income",
    "which earned": "net income",
    "op income": "operating income",
    "operating income": "operating income",
    "r&d": "r&d expense",
    "rnd": "r&d expense",
    "research": "r&d expense",
    "research and development": "r&d expense",
    "r&d est": "r&d expense (est.)",
    "r&d estimate": "r&d expense (est.)",
    "assets": "total assets",
    "liabilities": "total liabilities",
    "debt": "total liabilities",
    "equity": "shareholders' equity",
    "shareholder equity": "shareholders' equity",
    "cash equivalent": "cash & cash equivalents",
    "cash equivalents": "cash & cash equivalent",
    "cash equivalents": "cash & cash equivalents",
    "cash eq": "cash & cash equivalents",
    "cash equi": "cash & cash equivalents",
    "cash equiv": "cash & cash equivalents",
    "operating cash": "operating cash flow",
    "op cash": "operating cash flow",
    "operating cash flow": "operating cash flow",
    "ocf": "operating cash flow",
    "capex": "capital expenditures",
    "capital expense": "capital expenditures",
    "cap exp": "capital expenditures",
    "cap expenditure": "capital expenditures",
    "cap expenditures": "capital expenditures",
    "capital expenditure": "capital expenditures",
    "capital expenditures": "capital expenditure",
    "free cash flow": "free cash flow (est.)",
    "cash flow": "free cash flow (est.)",
    "estimated free cash": "free cash flow (est.)",
    "free cash flow est": "free cash flow (est.)",
    "est free cash flow": "free cash flow (est.)",
    "fcf": "free cash flow (est.)",
    "eps bas": "eps (basic)",
    "eps basic": "eps (basic)",
    "eps diluted": "eps dil",
    "eps (diluted)": "eps dil",
    "eps dil": "eps (diluted)",
    "eps diluted": "eps (diluted)",
    "cloud": "cloud revenue",
    "cloud revenue": "cloud revenue",
}

# Enhanced year phrase mapping
def extract_years(message):
    years = []
    message = message.lower()

    # Regex year detection
    year_matches = re.findall(r"\b(2022|2023|2024)\b", message)
    years.extend([int(y) for y in year_matches])

    # Phrase-based mappings
    if (
        "all years" in message or "last 3 years" in message or
        "since 2022" in message or "past 3 years" in message
    ):
        return [2022, 2023, 2024]
    if (
        "last two years" in message or "last 2 years" in message or
        "past 2 years" in message or "since 2023" in message or
        "second and third year" in message or "2nd and 3rd year" in message or "second and last year" in message
    ):
        return [2023, 2024]
    if (
        "first and last year" in message or "1st and 3rd year" in message
    ):
        return [2022, 2024]
    if (
        "first two years" in message or "first 2 years" in message or
        "1st and 2nd year" in message or "first and second year" in message
    ):
        return [2022, 2023]
    if (
        "second year" in message or "2nd year" in message
    ):
        return [2023]
    if (
        "first year" in message or "1st year" in message
    ):
        return [2022]
    if (
        "third year" in message or "3rd year" in message or
        "latest year" in message or "last year" in message or "2024" in message
    ):
        return [2024]

    return sorted(list(set(years))) if years else all_years

# Chatbot core
def handle_message(message, history):
    message = message.lower()

    # Companies
    companies = re.findall(r"\b(tesla|apple|microsoft)\b", message)
    if not companies or "all companies" in message or "compare companies" in message:
        companies = default_companies

    # Years
    years = extract_years(message)

    # Metric
    metric_match = [m for m in metrics_list if m in message]
    if not metric_match:
        for alias, standard in alias_dict.items():
            if re.search(rf"\b{re.escape(alias)}\b", message):
                metric_match = [standard]
                break
    metric = metric_match[0] if metric_match else None

    if not metric:
        reply = "❌ Please mention a valid financial metric (e.g. revenue, EPS, R&D)."
        return history + [{"role": "user", "content": message}, {"role": "assistant", "content": reply}], "", None

    # Filter Data
    filtered_df = df[
        (df['Metric'].str.lower() == metric) &
        (df['Company'].str.lower().isin(companies))
    ]

    if filtered_df.empty:
        reply = "❌ No matching data found for the given metric and companies."
        return history + [{"role": "user", "content": message}, {"role": "assistant", "content": reply}], "", None

    # Text Reply
    lines = [f"📊 **{metric.title()} Comparison**:"]
    for company in companies:
        sub = filtered_df[filtered_df["Company"].str.lower() == company]
        if sub.empty:
            continue
        lines.append(f"\n**{company.title()} – {metric.title()}**")
        for year in years:
            if str(year) in sub.columns:
                val = sub[str(year)].values[0]
                if pd.notna(val):
                    lines.append(f"{year}: ${val:,.0f}")
                else:
                    lines.append(f"{year}: Data not available")
            else:
                lines.append(f"{year}: Data not available")

    reply = "\n".join(lines)

    # Chart
    chart_data = []
    for _, row in filtered_df.iterrows():
        company = row["Company"]
        for year in years:
            val = row.get(str(year))
            if pd.notna(val):
                chart_data.append({
                    "Company": company,
                    "Year": year,
                    "Value": val
                })

    chart = None
    if chart_data:
        chart_df = pd.DataFrame(chart_data)
        chart = px.bar(
            chart_df,
            x="Year",
            y="Value",
            color="Company",
            barmode="group",
            title=f"{metric.title()} Comparison ({', '.join(map(str, years))})",
            text="Value"
        )
        chart.update_traces(texttemplate='%{text:.2s}', textposition='outside')
        chart.update_layout(template='plotly_white')

    # Final Output
    history = history + [
        {"role": "user", "content": message},
        {"role": "assistant", "content": reply}
    ]
    return history, "", chart

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown(
        """
        # 💬 Financial Chatbot  
        👨‍💻 Developed by: **📊 Bilal Hassan Nizami**  

        Ask about Tesla, Apple or Microsoft (Annual Financial Reports filed with US SEC, 2022–2024).  

        #### 📌 Example Questions:
        - *Compare R&D expenses for all companies*
        - *Tesla operating income all years*
        - *Apple vs Microsoft revenue since 2022*
        - *Who earned more in 1st and 3rd year?*
        """,
        elem_id="title"
    )

    with gr.Row():
        with gr.Column():
            msg = gr.Textbox(label="Ask your question:")
            btn = gr.Button("Submit")
            chatbot_output = gr.Chatbot(type="messages", label="Conversation")
        chart_output = gr.Plot(label="Chart")

    btn.click(
        fn=handle_message,
        inputs=[msg, chatbot_output],
        outputs=[chatbot_output, msg, chart_output],
    ).then(
        lambda chart: gr.update(visible=chart is not None),
        inputs=[chart_output],
        outputs=[chart_output],
    )

demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7865
* Running on public URL: https://da81e936a1a2478a84.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


**Key Findings**
Chatbot is much efficient now and handling company and year wise comparisons

# Making fixes in year detection and phrase base mapping 
# **& Having Final ROBUST code**

In [42]:
import pandas as pd
import re
import plotly.express as px
import gradio as gr

# Load data
try:
    df = pd.read_csv("chatbot_cleaned.csv")
except FileNotFoundError:
    raise FileNotFoundError("Could not find 'chatbot_cleaned.csv'. Ensure the file is in the root directory.")

# Metric cleaning
metrics_list = df['Metric'].str.lower().unique()
all_years = [2022, 2023, 2024]
default_companies = ["tesla", "apple", "microsoft"]

# Company aliases
company_aliases = {
    "tsla": "tesla",
    "ts": "tesla",
    "tsa": "tesla",
    "tesla": "tesla",
    
    "aapl": "apple",
    "appl": "apple",
    "ap": "apple",
    "aple": "apple",
    "apple": "apple",
    
    "ms": "microsoft",
    "msft": "microsoft",
    "mcsft": "microsoft",
    "msoft": "microsoft",
    "mcrsft": "microsoft",
    "mcrosft": "microsoft",
    "microsft": "microsoft",
    "microsoft": "microsoft"
}

# Metric aliases
alias_dict = {
    "revenue": "total revenue",
    "rvnu": "total revenue",
    "revnu": "total revenue",
    "revenu": "total revenue",
    "sales": "total revenue",
    "profit": "net income",
    "earnings": "net income",
    "earned": "net income",
    "earned more": "net income",
    "ernd": "net income",
    "earnd": "net income",
    "erned": "net income",
    "made profit": "net income",
    "made money": "net income",
    "make money": "net income",
    "made more": "net income",
    "make more": "net income",
    "who earned more": "net income",
    "which company earned more": "net income",
    "which earned": "net income",
    "op income": "operating income",
    "operating income": "operating income",
    "r&d": "r&d expense",
    "rnd": "r&d expense",
    "research": "r&d expense",
    "research and development": "r&d expense",
    "r&d est": "r&d expense (est.)",
    "r&d estimate": "r&d expense (est.)",
    "assets": "total assets",
    "ast": "total assets",
    "asset": "total assets",
    "asets": "total assets",
    "aset": "total assets",
    "liabilities": "total liabilities",
    "liab": "total liabilities",
    "liabiliti": "total liabilities",
    "liability": "total liabilities",
    "lia": "total liabilities",
    "debt": "total liabilities",
    "equity": "shareholders' equity",
    "shareholder equity": "shareholders' equity",
    "cash equivalent": "cash & cash equivalents",
    "cash equivalents": "cash & cash equivalents",
    "cash eq": "cash & cash equivalents",
    "cash equi": "cash & cash equivalents",
    "cash equiv": "cash & cash equivalents",
    "operating cash": "operating cash flow",
    "op cash": "operating cash flow",
    "operating cash flow": "operating cash flow",
    "ocf": "operating cash flow",
    "capex": "capital expenditures",
    "capital expense": "capital expenditures",
    "cap exp": "capital expenditures",
    "cap expenditure": "capital expenditures",
    "capital expenditures": "capital expenditures",
    "free cash flow": "free cash flow (est.)",
    "cash flow": "free cash flow (est.)",
    "estimated free cash": "free cash flow (est.)",
    "free cash flow est": "free cash flow (est.)",
    "est free cash flow": "free cash flow (est.)",
    "fcf": "free cash flow (est.)",
    "eps bas": "eps (basic)",
    "eps basic": "eps (basic)",
    "eps diluted": "eps (diluted)",
    "eps (diluted)": "eps (diluted)",
    "eps dil": "eps (diluted)",
    "cloud": "cloud revenue",
    "cloud revenue": "cloud revenue"
}

# Enhanced year phrase mapping
def extract_years(message):
    years = []
    message = message.lower()

    year_matches = re.findall(r"\b(2022|2023|2024)\b", message)
    years.extend([int(y) for y in year_matches])

    if any(phrase in message for phrase in ["all years", "last 3 years", "since 2022", "past 3 years"]):
        return [2022, 2023, 2024]
    if any(phrase in message for phrase in ["last two years", "last 2 years", "past 2 years", "since 2023", "second and third year", "2nd and 3rd year", "second and last year"]):
        return [2023, 2024]
    if any(phrase in message for phrase in ["first and last year", "1st and 3rd year"]):
        return [2022, 2024]
    if any(phrase in message for phrase in ["first two years", "first 2 years", "1st and 2nd year", "first and second year"]):
        return [2022, 2023]
    if any(phrase in message for phrase in ["second year", "2nd year"]):
        return [2023]
    if any(phrase in message for phrase in ["first year", "1st year"]):
        return [2022]
    if any(phrase in message for phrase in ["third year", "3rd year", "latest year", "last year", "2024"]):
        return [2024]

    return sorted(list(set(years))) if years else all_years

# Chart functions
def plot_bar_chart(company_df, metric, year):
    fig = px.bar(company_df, x='Company', y=str(year), title=f"{metric.title()} by Company in {year}", text=str(year))
    fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
    fig.update_layout(template='plotly_white', uniformtext_minsize=8, uniformtext_mode='show')
    return fig

def plot_comparison_chart(metric, years, companies):
    chart_data = []
    for _, row in df[
        (df['Metric'].str.lower() == metric) &
        (df['Company'].str.lower().isin(companies))
    ].iterrows():
        company = row["Company"]
        for year in years:
            val = row.get(str(year))
            if pd.notna(val):
                chart_data.append({
                    "Company": company,
                    "Year": year,
                    "Value": val
                })
    if not chart_data:
        return None
    chart_df = pd.DataFrame(chart_data)
    fig = px.bar(
        chart_df,
        x="Year",
        y="Value",
        color="Company",
        barmode="group",
        title=f"{metric.title()} Comparison ({', '.join(map(str, years))})",
        text="Value"
    )
    fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
    fig.update_layout(template='plotly_white')
    return fig

# Chatbot core
def handle_message(message, history):
    if not message:
        return history, "Please enter a question.", None

    message = message.lower()
    
    # Match company aliases
    raw_companies = re.findall(r"\b[a-z]{2,10}\b", message)
    companies = [company_aliases[c] for c in raw_companies if c in company_aliases]
    
    if not companies or any(phrase in message for phrase in ["all companies", "compare companies"]):
        companies = default_companies

    years = extract_years(message)

    metric_match = [m for m in metrics_list if m in message]
    if not metric_match:
        for alias, standard in alias_dict.items():
            if re.search(rf"\b{re.escape(alias)}\b", message):
                metric_match = [standard]
                break
    metric = metric_match[0] if metric_match else None

    if not metric:
        reply = "❌ Please mention a valid financial metric (e.g., revenue, EPS, R&D)."
        return history + [{"role": "user", "content": message}, {"role": "assistant", "content": reply}], "", None

    filtered_df = df[
        (df['Metric'].str.lower() == metric) &
        (df['Company'].str.lower().isin(companies))
    ]

    if filtered_df.empty:
        reply = "❌ No matching data found for the given metric and companies."
        return history + [{"role": "user", "content": message}, {"role": "assistant", "content": reply}], "", None

    lines = [f"📊 **{metric.title()} Comparison**:"]
    for company in companies:
        sub = filtered_df[filtered_df["Company"].str.lower() == company]
        if sub.empty:
            continue
        lines.append(f"\n**{company.title()} – {metric.title()}**")
        for year in years:
            if str(year) in sub.columns:
                val = sub[str(year)].values[0] if not sub[str(year)].empty else None
                if pd.notna(val):
                    lines.append(f"{year}: ${val:,.0f}")
                else:
                    lines.append(f"{year}: Data not available")
            else:
                lines.append(f"{year}: Data not available")

    reply = "\n".join(lines)
    chart = plot_comparison_chart(metric, years, companies)

    history = history + [
        {"role": "user", "content": message},
        {"role": "assistant", "content": reply}
    ]
    return history, "", chart

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown(
        """
        # 💬 Financial Chatbot  
        👨‍💻 Developed by: **📊 Bilal Hassan Nizami**  

        Ask about Tesla, Apple, or Microsoft (Annual Financial Reports filed with US SEC, 2022–2024).  

        #### 📌 Example Questions:
        - *Compare R&D expenses for all companies*
        - *Tesla operating income all years*
        - *Apple vs Microsoft revenue since 2022*
        - *Who earned more in 1st and 3rd year?*
        """,
        elem_id="title"
    )

    with gr.Row():
        with gr.Column():
            msg = gr.Textbox(label="Ask your question:")
            btn = gr.Button("Submit")
            chatbot_output = gr.Chatbot(type="messages", label="Conversation")
        chart_output = gr.Plot(label="Chart")

    btn.click(
        fn=handle_message,
        inputs=[msg, chatbot_output],
        outputs=[chatbot_output, msg, chart_output],
    ).then(
        lambda chart: gr.update(visible=chart is not None),
        inputs=[chart_output],
        outputs=[chart_output],
    )

# Launch
demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7875
* Running on public URL: https://7119f9ec7e36607a49.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Key Findings:
**Code is now much robust in handling company & year wise comparison with bar chart visualization to help undertand trend better. It is handling typo error also which will make UX seamless**